In [13]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [14]:
def dbPath() -> Path:
    cwd = Path.cwd()
    dc2 = cwd.parent
    db = dc2.joinpath("data/database_final.db")
    return db

In [15]:
def modelPath(lsoaCode:str) ->Path:
    cwd = Path.cwd()
    dc2 = cwd.parent
    modelPath = dc2.joinpath(f"data/models/{lsoaCode}.bin")
    return modelPath

In [16]:
conn = sqlite3.connect(dbPath())

In [17]:
query_lsoa_codes = "SELECT geogcode FROM lsoa_code_to_name"
lsoa_codes = pd.read_sql(query_lsoa_codes, conn)


In [18]:
query_unemployement = "SELECT * FROM montly_unemployement_claimant_count_by_lsoa_barnet"
unemployement = pd.read_sql(query_unemployement, conn)
unemployement.dropna(inplace=True)
unemployement.drop(columns="index", inplace=True)
unemployement = unemployement[(unemployement["date"]<"2020") & (unemployement["date"]>"2012")]


In [19]:
df = pd.DataFrame(columns=lsoa_codes["geogcode"].to_list(), index=unemployement["date"].unique())
for row in tqdm(unemployement.index):
    df[unemployement["geogcode"][row]][unemployement["date"][row]] = unemployement["value"][row]

for code in tqdm(lsoa_codes["geogcode"].to_list()):
    for i in range(12):
        df[f"{code}_shift_{i+1}"] = df[f"{code}"].shift(i+1)

  0%|          | 0/17808 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

/tmp/ipykernel_7886/2313649989.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{code}_shift_{i+1}"] = df[f"{code}"].shift(i+1)
/tmp/ipykernel_7886/2313649989.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{code}_shift_{i+1}"] = df[f"{code}"].shift(i+1)
/tmp/ipykernel_7886/2313649989.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a

In [20]:
mseDct = {}
modelDct = {}
df = df.dropna()
for col in tqdm(lsoa_codes["geogcode"].to_list()):
    y = df[col].copy()
    x = df.drop(col, axis=1).copy()
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.8)
    model = xgb.XGBRegressor()
    # dmx_x_train = xgb.DMatrix(X_train.values)
    # dmx_x_test= xgb.DMatrix(X_test.values)
    # dmx_y_train= xgb.DMatrix(y_train.values)
    # dmx_y_test= xgb.DMatrix(y_test.values)
    model.fit(X_train.values, y_train)
    y_pred = model.predict(X_test.values)
    mseDct[col] = mean_squared_error(y_test, y_pred)
    model.save_model( modelPath(col))

  0%|          | 0/212 [00:00<?, ?it/s]

In [21]:
mseDct

{'Column Total': 96680.39386720904,
 'E01000115': 9.481705057250016,
 'E01000116': 9.833694204487395,
 'E01000117': 13.641690674550773,
 'E01000118': 13.368692879382126,
 'E01000119': 13.361804948531745,
 'E01000120': 21.120222713428795,
 'E01000121': 17.9478466054355,
 'E01000122': 17.222551722634435,
 'E01000123': 20.277081034298824,
 'E01000124': 59.35155223381875,
 'E01000125': 36.2534757064242,
 'E01000126': 130.44359628078232,
 'E01000127': 20.288043326612687,
 'E01000128': 52.39905524290142,
 'E01000129': 21.223346989554116,
 'E01000130': 37.772371278664025,
 'E01000131': 18.873063639840296,
 'E01000132': 20.95514370699269,
 'E01000133': 36.41319344143285,
 'E01000134': 18.57398682058606,
 'E01000135': 14.496811727048946,
 'E01000136': 4.3098470012637415,
 'E01000137': 56.9848649365143,
 'E01000138': 49.68012612449814,
 'E01000139': 17.301150478216297,
 'E01000140': 16.021452016646467,
 'E01000141': 49.587469800120495,
 'E01000142': 34.523119706675246,
 'E01000143': 30.137108208